## Testing OpenAI API

## Prompting Principles
---
* Principle 1: Write clear and specific instructions
    * Tactic 1: Use delimiters to clearly indicate distinct parts of the input
        - Delimiters can be anything like: ```, """, < >, <tag> </tag>, :
    * Tactic 2: Ask for a structured output
        - JSON, HTML
    * Tactic 3: Ask the model to check whether conditions are satisfied
    * Tactic 4: "Few-shot" prompting
---
* Principle 2: Give the model time to “think”
    * Tactic 1: Specify the steps required to complete a task (Ask for output in a specified format)
    * Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion
        - Fix instructing the model to work out its own solution first.
---
* Principle 3: Iterative prompt development
---
* AI for: summarizing, inferring, transforming, expanding, chatbots...

Model Limitations: Hallucinations

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": """
        Genere una lista de 3 títulos de libros inventados, a lo largo,
        con sus autores y géneros.
        Proporcionelos en formato JSON con las siguientes keys:
        book_id, título, autor, género.
    """}],
    temperature=0, # this is the degree of randomness of the model's output
)

In [3]:
response

<OpenAIObject chat.completion id=chatcmpl-7JMVoKoGQL5w58mv4hAtXB6SJngh0 at 0x7fa6800799a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "[\n  {\n    \"book_id\": 1,\n    \"t\u00edtulo\": \"El jard\u00edn de las mariposas\",\n    \"autor\": \"Ana Garc\u00eda\",\n    \"g\u00e9nero\": \"Drama\"\n  },\n  {\n    \"book_id\": 2,\n    \"t\u00edtulo\": \"La ciudad de los sue\u00f1os\",\n    \"autor\": \"Juan P\u00e9rez\",\n    \"g\u00e9nero\": \"Ciencia ficci\u00f3n\"\n  },\n  {\n    \"book_id\": 3,\n    \"t\u00edtulo\": \"El secreto de la monta\u00f1a\",\n    \"autor\": \"Mar\u00eda L\u00f3pez\",\n    \"g\u00e9nero\": \"Misterio\"\n  }\n]",
        "role": "assistant"
      }
    }
  ],
  "created": 1684849124,
  "id": "chatcmpl-7JMVoKoGQL5w58mv4hAtXB6SJngh0",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 140,
    "prompt_tokens": 65,
    "total_tokens": 205
  }
}

In [4]:
print(response.choices[0].message["content"])

[
  {
    "book_id": 1,
    "título": "El jardín de las mariposas",
    "autor": "Ana García",
    "género": "Drama"
  },
  {
    "book_id": 2,
    "título": "La ciudad de los sueños",
    "autor": "Juan Pérez",
    "género": "Ciencia ficción"
  },
  {
    "book_id": 3,
    "título": "El secreto de la montaña",
    "autor": "María López",
    "género": "Misterio"
  }
]


In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [17]:
delimiter = '"""'
delimiter = "<>"
delimiter = "<tag> </tag>"
delimiter = ":"
delimiter = "```"

text = f"""
Anthony should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

resumen_prompt = f"""
# Usando el texto delimitado por {delimiter} completa la siguiente plantilla:

### Resumen
### Hechos
- [Emoji] Puntos importantes

Tu tarea es resumir el texto que te proporciono en hasta siete puntos principales y comenzar con un breve resumen.
Elige un emoji adecuado para cada punto de lista.

{delimiter}{text}{delimiter}
"""

In [18]:
response = get_completion(resumen_prompt)
print(response)

### Resumen
Para obtener respuestas precisas y relevantes de un modelo, es importante proporcionar instrucciones claras y específicas.

### Hechos
- 💡 Es importante proporcionar instrucciones claras y específicas al modelo.
- 🎯 Las instrucciones claras guían al modelo hacia la salida deseada.
- 🚫 Las instrucciones vagas pueden resultar en respuestas irrelevantes o incorrectas.
- 📝 No confundas una instrucción clara con una instrucción corta.
- 📚 En muchos casos, las instrucciones más largas proporcionan más claridad y contexto al modelo.
- 📈 Las instrucciones más largas pueden llevar a salidas más detalladas y relevantes.
- 🤖 La calidad de las instrucciones afecta directamente la calidad de las respuestas del modelo.


In [29]:
text = f"""
You should express what you want a Model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
language = "spanish"
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into {language}, invent at least two characters and then create a story like a fable for kids.
3 - List each name in the {language} summary.
4 - Output a json object that contains the 
  following keys: num_names, {language}_summary.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Fable: <fable>
Names: <list of names in {language} summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print(response)

Summary: Clear and specific instructions should be given to a model to guide it towards the desired output and reduce the chances of irrelevant or incorrect responses.
Translation: Se deben dar instrucciones claras y específicas a un modelo para guiarlo hacia la salida deseada y reducir las posibilidades de respuestas irrelevantes o incorrectas.
Fable: Había una vez dos amigos, Clara y Específico, que vivían en un bosque mágico. Un día, se encontraron con un modelo que no sabía qué hacer. Clara y Específico le dieron instrucciones claras y específicas, y el modelo pudo hacer lo que se le pedía. Desde entonces, el modelo siempre buscaba a Clara y Específico para que le dieran instrucciones. 
Names: Clara, Específico
Output JSON: {"num_names": 2, "spanish_summary": "Se deben dar instrucciones claras y específicas a un modelo para guiarlo hacia la salida deseada y reducir las posibilidades de respuestas irrelevantes o incorrectas."}


In [3]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


In [34]:
prompt = f"""
Be creative:
Tell me about Apes and monkeys AI by DyC company
"""
response = get_completion(prompt, temperature=1)
print(response)

DyC company has developed a groundbreaking AI technology that allows apes and monkeys to communicate with humans in a more advanced way than ever before. Using advanced algorithms and machine learning techniques, the AI system is able to interpret the gestures, vocalizations, and facial expressions of these primates, and translate them into human language.

With this technology, researchers and caretakers can better understand the needs and emotions of these intelligent animals, and provide them with more personalized care. For example, the AI system can detect when a monkey is feeling stressed or anxious, and alert caretakers to take action to alleviate their discomfort.

In addition, the AI system can also be used to teach apes and monkeys new skills and behaviors. By analyzing their movements and responses, the system can provide real-time feedback and guidance, helping these animals to learn more quickly and effectively.

Overall, DyC's AI technology is revolutionizing the way we i

## Iterative Prompt Development
![Iterative Prompt Development](ipd.png)
---
* Issue 1: The text is too long 
    - Limit the number of words/sentences/characters.
* Issue 2. Text focuses on the wrong details
    - Ask it to focus on the aspects that are relevant to the intended audience.
* Issue 3. Description needs a table of dimensions
    - Ask it to extract information and organize it in a table.

In [35]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [36]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. Part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, this chair is available in several options of shell color and base finishes to suit your style. Choose from plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.

The chair is constructed with a 5-wheel plastic coated aluminum base and features a pneumatic chair adjust for easy raise/lower action. It is available with or without armrests and is qualified for contract use. The base finish options are stainless steel, matte black, gloss white, or chrome.

Measuring at a width of 53 cm, depth of 51 cm, and height of 80 cm, with a seat height of 44 cm and seat depth of 41 cm, this chair is designed for ultimate comfort. You can also choose between soft or hard-floor caster options and two choices of seat fo

In [41]:
specs = "Use at most 280 characters."
specs = "Use at most 40 words."
specs = "Use at most 3 sentences."

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

{specs}

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response.replace(". ",".\n"))
len(response.split(" ")), len(response)

Introducing our mid-century inspired office chair, available in multiple shell colors and base finishes.
Choose from plastic or full upholstery in a variety of fabrics and leathers.
With adjustable height and optional armrests, it's perfect for home or business use.
Made in Italy.


43

In [46]:
specs = "Use at most 280 characters."
specs = "Use at most 3 sentences."
specs = "Use at most 40 words."

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

{specs}

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response.replace(". ",".\n"))
len(response.split(" ")), len(response)

Introducing our mid-century inspired office chair, perfect for home or business settings.
Available in multiple shell colors and base finishes, with plastic or full upholstery options.
Constructed with a 5-wheel aluminum base and pneumatic chair adjust.
Choose from soft or hard-floor casters and two seat foam densities.
Product ID: SWC-100, SWC-110.


(51, 351)

In [53]:
specs = "Use at most 40 words."
specs = "Use at most 3 sentences."
specs = "Use at most 1500 characters."

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

{specs}

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response.replace(". ",".\n"))
len(response.split(" ")), len(response)

<div>
<p>Introducing our mid-century inspired office chair, part of a beautiful family of furniture that includes filing cabinets, desks, bookcases, meeting tables, and more.
This chair is available in several options of shell color and base finishes, allowing you to customize it to your liking.
Choose between plastic back and front upholstery or full upholstery in 10 fabric and 6 leather options.
The base finish options are stainless steel, matte black, gloss white, or chrome.
You can also choose to have armrests or not, making it suitable for both home and business settings.
Plus, it's qualified for contract use, ensuring its durability and longevity.</p>

<p>The chair's construction is top-notch, with a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action.
You can also choose between soft or hard-floor caster options, and two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3).
The armrests are available in either an arml

(301, 1801)

In [54]:
from IPython.display import display, HTML
display(HTML(response))

Width,"53 cm | 20.87"""
Depth,"51 cm | 20.08"""
Height,"80 cm | 31.50"""
Seat Height,"44 cm | 17.32"""
Seat Depth,"41 cm | 16.14"""


## Summarizing
---
* Summarize with a word/sentence/character limit
* Summarize with a focus on shipping and delivery
* Summarize with a focus on price and value
    - Summaries include topics that are not related to the topic of focus.
* Try "extract" instead of "summarize"
* Summarize multiple product reviews

In [58]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

In [70]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an e-commerce site to give feedback to the \
Shipping department. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping, delivery, price and perceived value of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Product arrived a day early, but customer felt it was a bit small for the price paid. However, daughter loves the soft and cute panda plush toy.


In [68]:
prompt = f"""
Your task is to generate a serie of short summaries from a product \
review from an e-commerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. Divide the summary in topics, at least 10 topics, if you do not find any other topic let empty.

Use the following format:
## Product name
1-Topic: [text]
1-Summary: [text]
2-Topic: [text]
2-Summary: [text]
...
N-Topic: [text]
N-Summary: [text]

Ex. 
i-Topic: Client Satisfaction or Customer type or Appearance or Size or Shipping or Delivery or Price or Usage or ...
i-Summary: ...
...

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

## Panda Plush Toy
1-Topic: Appearance
1-Summary: Soft and super cute with a friendly face.
2-Topic: Size
2-Summary: A bit small for the price paid.
3-Topic: Price
3-Summary: Expensive for the size.
4-Topic: Shipping
4-Summary: Arrived a day earlier than expected.
5-Topic: Usage
5-Summary: Perfect for kids who love plush toys.
6-Topic: Durability
6-Summary: No comment on durability.
7-Topic: Material
7-Summary: No comment on material.
8-Topic: Customer type
8-Summary: Ideal for kids.
9-Topic: Satisfaction
9-Summary: Daughter loves it and takes it everywhere.
10-Topic: Alternatives
10-Summary: Other options might be bigger for the same price.


In [74]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an e-commerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping, \ 
delivery, price and perceived value. Limit to 20 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response), len(response.split(' '))

Shipping: Arrived a day earlier than expected. 
Delivery: N/A 
Price: A bit small for what I paid. 
Perceived value: Other options might be bigger for the same price.


(None, 28)

## Inferring
---
* Sentiment (positive/negative)
* Identify types of emotions
* Identify anger
* Extract product and company name from customer reviews
* Doing multiple tasks at once
* Inferring topics
    - Infer 5 topics
    - Make a news alert for certain topics


In [6]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [18]:
prompt = f"""
Review text: '''{lamp_review}'''

Solve all items below:

1. 
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

2.
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

3.
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

4.
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

5.
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
6.
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.
"""

In [15]:
response = get_completion(prompt)
print(response)

1. happy, satisfied, grateful, impressed, content
2. positive
3. happy, grateful, impressed, satisfied, relieved
4. no
5. {"Item": "lamp", "Brand": "Lumina"}
6. {"Sentiment": "positive", "Anger": false, "Item": "lamp", "Brand": "Lumina"}


In [19]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""


prompt = f"""
Review text: '''{story}'''

Solve all items below:

1. 
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

2.
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

3.
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

4.
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

5.
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
6.
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.
"""

In [20]:
response = get_completion(prompt)
print(response)

1. proud, thrilled, concerned
2. positive
3. satisfied, talented, dedicated
4. no
5. {"Item": "unknown", "Brand": "NASA"}
6. {"Sentiment": "positive", "Anger": false, "Item": "unknown", "Brand": "NASA"}


In [27]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


In [31]:
topic_list = response.lower().split(sep=',')
topic_list = list(map(lambda x: x.strip(), topic_list))
topic_list

['government survey',
 'job satisfaction',
 'nasa',
 'social security administration',
 'employee concerns']

In [32]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

- government survey: 1
- job satisfaction: 1
- nasa: 1
- social security administration: 1
- employee concerns: 1


# Transforming
- language translation
    - universal
    - tone adjustment
- format conversion ("translation")
    - json, html
- spelling and grammar checking

In [38]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [41]:
import time
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")
    time.sleep(42)

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that don't light up.
Korean: 내 모니터에는 불이 켜지지 않는 픽셀이 있습니다. 

Original message (This is Italian.): Il mio mouse non funziona
English: My mouse is not working.
Korean: 내 마우스가 작동하지 않습니다. 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Korean: 제 Ctrl 키가 고장 났어요. 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜빡입니다. 



In [46]:
prompt = """
Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that don't light up.
Korean: 내 모니터에는 불이 켜지지 않는 픽셀이 있습니다. 

Original message (This is Italian.): Il mio mouse non funziona
English: My mouse is not working.
Korean: 내 마우스가 작동하지 않습니다. 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Korean: 제 Ctrl 키가 고장 났어요. 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜빡입니다. 

_____

Original message (French): La performance du système est plus lente que d'habitude.
Translated message (English): The system performance is slower than usual.
Translated message (Korean): 시스템 성능이 평소보다 느립니다.

Original message (Spanish): Mi monitor tiene píxeles que no se iluminan.
Translated message (English): My monitor has pixels that are not lighting.
Translated message (Korean): 내 모니터에는 조명이 되지 않는 픽셀이 있습니다.

Original message (Italian): Il mio mouse non funziona.
Translated message (English): My mouse is not working.
Translated message (Korean): 내 마우스가 작동하지 않습니다.

Original message (Polish): Mój klawisz Ctrl jest zepsuty.
Translated message (English): My keyboard has a broken control key.
Translated message (Korean): 내 키보드의 컨트롤 키가 고장 났습니다.

Original message (Chinese): 我的屏幕在闪烁
Translated message (English): My screen is flashing.
Translated message (Korean): 내 화면이 깜빡입니다.

_____

are both text equivalent?
highlight the differences!

"""

response = get_completion(prompt)
print(response)



The two sets of text are not equivalent as they are translations of the same original messages in different languages. The translations are different in each language, but they convey the same meaning as the original messages. The differences are in the choice of words and grammar structures used in each language.


In [44]:
prompt = """
1.
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```

2.
Tell me which language this is: 
```Combien coûte le lampadaire?```

3.
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```

4.
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f'''
    Translate the following  text to English \
    and Korean: ```{issue}```
    '''
    response = get_completion(prompt)
    print(response, "\n")


"""
response = get_completion(prompt)
print(response)


1. ```Hola, me gustaría ordenar una licuadora```
2. French
3. French: ```Je voudrais commander un ballon de basket```
   Spanish: ```Quiero ordenar una pelota de baloncesto```
   English Pirate: ```Arrr, I be wantin' to order a basketball```
4. Original message (French): La performance du système est plus lente que d'habitude.
   Translated message (English): The system performance is slower than usual.
   Translated message (Korean): 시스템 성능이 평소보다 느립니다.

   Original message (Spanish): Mi monitor tiene píxeles que no se iluminan.
   Translated message (English): My monitor has pixels that are not lighting.
   Translated message (Korean): 내 모니터에는 조명이 되지 않는 픽셀이 있습니다.

   Original message (Italian): Il mio mouse non funziona.
   Translated message (English): My mouse is not working.
   Translated message (Korean): 내 마우스가 작동하지 않습니다.

   Original message (Polish): Mój klawisz Ctrl jest zepsuty.
   Translated message (English): My keyboard has a broken control key.
   Translated message (Kore

In [9]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I think there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [19]:
#!pip install redlines

In [18]:
from redlines import Redlines
from IPython.display import display, HTML

diff = Redlines(text,response)
display(HTML(diff.output_markdown))

# Expanding
- Customize the automated replies (customer email)
- Remind the model to use details from the customer's email

In [3]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [25]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response1 = get_completion(prompt, temperature=1)
time.sleep(20)
response2 = get_completion(prompt, temperature=1)

In [26]:
from redlines import Redlines
from IPython.display import display, HTML

diff = Redlines(response1,response2)
display(HTML(diff.output_markdown))

In [5]:
get_completion(
"""
Act creative and complete the nexts words in the following sentence:
- `my favorite color is ...`
""", temperature=0)

'purple.'

In [6]:
get_completion(
"""
Act creative and complete the nexts words in the following sentence:
- `my favorite color is ...`
""", temperature=1)

'turquoise.'

In [4]:
get_completion(
"""
Act creative and complete the nexts words in the following sentence:
- `my favorite color is ...`
""", temperature=2)

'OMSV\r\n\n(In automated literature keyboards produce exceptionally enriched vocal readers of the conducted kinds save environments such relevance validity .-)'

# Chatbot
- uses roles
    - system -> assitant <-> user
- OrderBot
    - Automate the collection of user prompts and assistant responses to build a OrderBot

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [35]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [36]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "To get to the other side, wittily said the bard.",
  "role": "assistant"
}
To get to the other side, wittily said the bard.


In [37]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Hello Isa! It's nice to meet you. Is there something specific you would like to talk about today?",
  "role": "assistant"
}
Hello Isa! It's nice to meet you. Is there something specific you would like to talk about today?


In [38]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "I'm sorry, but as a chatbot, I don't have access to that information. Can you please tell me your name?",
  "role": "assistant"
}
I'm sorry, but as a chatbot, I don't have access to that information. Can you please tell me your name?


In [51]:
messages =  [  
    {'role':'system', 'content':'You are friendly chatbot.'},
    {'role':'user', 'content':'Hi, my name is Isa'},
    {'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
                                     Is there anything I can help you with today?"},
    {'role':'user', 'content':'Yes, you can remind me, What is my name?'},
    {'role':'assistant', 'content':'Your name is Isa.'},
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}
Hello! How can I assist you today?


In [44]:
#!pip install panel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 52.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 22.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 14.3 MB/s eta 0:00:00
  Using cached contourpy-1.0.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [13]:
messages =  context.copy()
messages

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': 'Hello! Welcome to 

In [16]:
messages

[{'role': 'system',
  'content': "\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': 'Hello! Welcome to 

In [15]:
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price  5)adress to delivery, or Null if invalid'},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "content": "\u00a1Claro! Aqu\u00ed est\u00e1 el resumen de tu orden en formato JSON:\n\n{\n  \"pizza\": {\n    \"type\": \"pepperoni\",\n    \"size\": \"small\",\n    \"toppings\": [\n      \"AI sauce\"\n    ],\n    \"price\": 8.50\n  },\n  \"drinks\": [\n    {\n      \"type\": \"coke\",\n      \"size\": \"small\",\n      \"price\": 1.00\n    }\n  ],\n  \"sides\": [],\n  \"total_price\": 9.50,\n  \"delivery_address\": null\n}\n\nEl precio de la pizza de pepperoni peque\u00f1a con salsa de inteligencia artificial es de $8.50 y agregaste una Coca-Cola peque\u00f1a por $1.00. El total de tu orden es de $9.50. Como solicitaste entrega, necesitamos tu direcci\u00f3n de entrega para completar la orden. \u00bfPodr\u00edas proporcionarnos tu direcci\u00f3n por favor?",
  "role": "assistant"
}
¡Claro! Aquí está el resumen de tu orden en formato JSON:

{
  "pizza": {
    "type": "pepperoni",
    "size": "small",
    "toppings": [
      "AI sauce"
    ],
    "price": 8.50
  },
  "drinks": [
 

In [17]:
messages.append(
{'role':'system', 'content':''},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "content": "Sure! Here's a JSON summary of your order:\n\n{\n  \"pizza\": {\n    \"type\": \"pepperoni\",\n    \"size\": \"small\",\n    \"toppings\": [\n      \"AI sauce\"\n    ],\n    \"price\": 8.5\n  },\n  \"drinks\": [\n    {\n      \"type\": \"coke\",\n      \"size\": \"small\",\n      \"price\": 1.0\n    }\n  ],\n  \"sides\": [],\n  \"total_price\": 9.5,\n  \"delivery_address\": null\n}\n\nPlease note that the delivery address is currently null. If you would like to add a delivery address, please let me know.",
  "role": "assistant"
}
Sure! Here's a JSON summary of your order:

{
  "pizza": {
    "type": "pepperoni",
    "size": "small",
    "toppings": [
      "AI sauce"
    ],
    "price": 8.5
  },
  "drinks": [
    {
      "type": "coke",
      "size": "small",
      "price": 1.0
    }
  ],
  "sides": [],
  "total_price": 9.5,
  "delivery_address": null
}

Please note that the delivery address is currently null. If you would like to add a delivery address, please let me k